Goal: regressions that work with deductibles (which might be overly amibtious). 

Basically, we have a claim count, which is a Poisson model. Every claim will come from a gamma distribtuion. However, if it is under a certain deductible, we do not code it in. 

Couple of imagined steps: 
- do a Poisson, where claim count is generated randomly, but every claim is ignored with X% probability (this is probably super easy)
- same as above, but make the X% probability depend on a variable, e.g., for every 1k increase, the chance of observing decreas
- make a Gamma with deductible
- combine everything, features determine baseline frequency, then the severity is clipped by a deductible amount, which then might impact frequency

Found some stuff like this one: https://www.stata.com/stata14/censored-poisson/ but it's not the type of censored I am looking for. 

Maybe censored is the wrong word...

In [39]:
import numpy as np
import pandas as pd

from sklearn.linear_model import PoissonRegressor, GammaRegressor

from scipy.special import factorial # for elementwise factorial calc of array

from functions import poisson_likelihood

In [3]:
%load_ext autoreload
%autoreload 2

# Frequency with Fixed Random Claim Cancel

One binary categorical feature, and each claim is cancelled by a fix percentage. The cancelled ones we don't observe. 

My theory is that this is simply going to be another Poisson where we multiply the lambda with the prob of cancel. Or 1 - prob of cancel...

In [3]:
# parameters
cancel_prob = 0.3 # each claim has a chance to not materialise

In [4]:
df = pd.DataFrame.from_dict({'cat_feature' : np.repeat([0,1],100000)})
df['poisson_lambda'] = df.apply(lambda x: 0.2 if x['cat_feature'] == 0 else 0.7, axis = 1)
df['raw_claim_number'] = np.random.poisson(df['poisson_lambda'])

Every raw_claim_number can be cancelled with the same chance. I think I want to generate this from scratch... If a row has a positive raw_claim_number, we want to generate that many random uniform variables, and put them in a list. And those lists will be in the next column. 

In [5]:
df['claim_cancellation'] = df.apply(lambda x: list(np.random.uniform(0,1,int(x['raw_claim_number']))), axis = 1)
df['claim_number'] = df.apply(lambda x: np.sum(cancel_prob <= np.array(x['claim_cancellation'])), axis = 1)

## Empirical Analysis

Did the censored claim count become Poisson with lambda that is original * chance of not canceling? 

In [5]:
empirical = pd.DataFrame(df[df['cat_feature'] == 1]['claim_number'].value_counts()).reset_index()
empirical['claim_number'] = empirical['claim_number'] / sum(empirical['claim_number'])
empirical.columns = ["value", "prob"]

In [14]:
empirical['theoretical_prob'] = empirical.apply(
    lambda x: poisson_likelihood(x["value"], 0.7 * (1-cancel_prob)), axis = 1)

In [15]:
empirical

,value,prob,theoretical_prob
0,0,0.61557,0.612626
1,1,0.29879,0.300187
2,2,0.07267,0.073546
3,3,0.01143,0.012012
4,4,0.00143,0.001472
5,5,0.00010,0.000144
6,6,0.00001,0.000012


It doesn't look wildly off...

## Math

Proving the above formulaically.

Make things a bit more formal. Let $X$ be our original random Poisson variable with parameter $\lambda$. $X$ represents the underlying number of incidents, the total ones, even the ones not reported as claims. Then $Y$ is generated from $X$ by randomly cancelling each incident with probability $p _{c}$, so $Y$ represent the materialised claim numbers. 

There are a number of important assumptions here: 
- each incident is 'cancelled' with exactly the same probability
- the cancellations are independent of each other
- probably others I can't think of now...

With these notations, what we are trying to prove here is that $Y$ is also a Poisson distribution. Let's start!

$Pr(Y = k) = Pr$(there remained exactly $k$ uncannelled incidents)

Let's think about the right side. In order for $k$ incidents to remain, the original $X$ variable had to be at least $k$. Then the cancellations follow a neg binomial distribution with parameter $p _{c}$. 

$Pr(Y = k) = $

$\sum \limits _{i=k} ^{\infty} \displaystyle \frac{\lambda ^ {i} e ^ {- \lambda}} {i!} \displaystyle \frac {i!}{k! (i - k)!} p _{c} ^{(i - k)} (1 - p _{c})^{k} = $

$\displaystyle \frac {e ^ {-\lambda} (1 - p _{c}) ^{k}}  {k!} \sum \limits _{i = k} ^{\infty} \displaystyle \frac {\lambda ^ {k} p _{c} ^ {(i - k)}} {(i - k)!} = $

$\displaystyle \frac {e ^ {-\lambda} (1 - p _{c}) ^{k}}  {k!} \lambda ^{k} \sum \limits _{i = 0} ^{\infty} \displaystyle \frac {\lambda ^{i} p_{c} ^{i}} {i!} = $

(using the sum form of $e^{x}$)

$\displaystyle \frac {e ^ {-\lambda} (\lambda(1 - p _{c})) ^{k}}  {k!} e ^{\lambda p _{c}} = $

$\displaystyle \frac {e ^ {-\lambda (1 - p _{c})} (\lambda (1 - p _{c}) ^{k}} {k!}  $

This is true for all $k$, (might not be derived from the formula, but it also works with 0), which means that $Y$ must be from a Poisson distribution with parameter $\lambda (1 - p _{c})$.

## GLM

Question: can we fit a GLM based on that feature? The Poisson lambda is linear by the $p _{c}$, so we can either set it as an offset or put in the logarithm as a feature. 

### Offset Method

In [15]:
df['cancel_prob'] = cancel_prob

In [24]:
mdl = PoissonRegressor(alpha = 0, max_iter = 10000)
X = df[['cat_feature']]
y = df['claim_number'] / (1 - df['cancel_prob'])
mdl.fit(X = X, y = y, sample_weight = (1-df['cancel_prob']))
df['pred_claim_number_v1'] = mdl.predict(X) * (1 - df['cancel_prob'])

In [25]:
df.sample(5)

,cat_feature,poisson_lambda,raw_claim_number,claim_cancellation,claim_number,pred_claim_number_v1,cancel_prob
113905,1,0.7,0,[],0,0.48859,0.3
48927,0,0.2,0,[],0,0.14027,0.3
48383,0,0.2,0,[],0,0.14027,0.3
193399,1,0.7,2,"[0.3313033847599537, 0.7846054377887441]",2,0.48859,0.3
194020,1,0.7,0,[],0,0.48859,0.3


OK, that's waht we wanted. (The two new predictions should be original poisson_lambda * (1 - prob cancel)). 

### Cancellation Prob as Feature

In [28]:
df['remains_prob'] = 1 - df['cancel_prob']
df['log_remains_prob'] = np.log(df['remains_prob'])

In [29]:
mdl = PoissonRegressor(alpha = 0, max_iter = 10000)
X = df[['cat_feature', 'log_remains_prob']]
y = df['claim_number']
mdl.fit(X = X, y = y)
df['pred_claim_number_v2'] = mdl.predict(X)

In [30]:
df.sample(5)

,cat_feature,poisson_lambda,raw_claim_number,claim_cancellation,claim_number,pred_claim_number_v1,cancel_prob,remains_prob,log_remains_prob,pred_claim_number_v2
44109,0,0.2,1,[0.7239890745792845],1,0.14027,0.3,0.7,-0.356675,0.140247
73169,0,0.2,0,[],0,0.14027,0.3,0.7,-0.356675,0.140247
157635,1,0.7,3,"[0.9531545247371701, 0.7912971045459187, 0.261...",2,0.48859,0.3,0.7,-0.356675,0.488767
131717,1,0.7,0,[],0,0.48859,0.3,0.7,-0.356675,0.488767
62524,0,0.2,1,[0.020591402671246817],0,0.14027,0.3,0.7,-0.356675,0.140247


In [31]:
mdl.coef_

array([1.2484801 , 0.25546392])

In [32]:
mdl.intercept_

-1.873231173251297

In [36]:
np.exp(mdl.intercept_ + mdl.coef_[0] + np.log(1-cancel_prob) * mdl.coef_[1])

0.48876735894841045

OK, I think there are perfectly correlated features, like the cancel_prob with intercept, so the coefs are kinda meaningless now. 

# Frequency with Changing Claim Cancel Prob

This should be quick, basically want to make $p _{c}$ a random variable between 0 and 1 and see if the methods above work. Going to keep everything else unchanged from previous section data. 

In [38]:
n_size = 200000

In [41]:
df = pd.DataFrame.from_dict({'cat_feature' : np.repeat([0,1],n_size / 2)})
df['poisson_lambda'] = df.apply(lambda x: 0.2 if x['cat_feature'] == 0 else 0.7, axis = 1)
df['raw_claim_number'] = np.random.poisson(df['poisson_lambda'])

In [42]:
df['cancel_prob'] = np.random.uniform(0.01, 0.99, n_size)

In [43]:
df['claim_cancellation'] = df.apply(lambda x: list(np.random.uniform(0,1,int(x['raw_claim_number']))), axis = 1)
df['claim_number'] = df.apply(lambda x: np.sum(np.array(x['cancel_prob']) <= np.array(x['claim_cancellation'])), axis = 1)

In [47]:
df['remains_prob'] = 1 - df['cancel_prob']
df['log_remains_prob'] = np.log(df['remains_prob'])

In [50]:
mdl = PoissonRegressor(alpha = 0, max_iter = 10000)
X = df[['cat_feature', 'log_remains_prob']]
y = df['claim_number']
mdl.fit(X = X, y = y)
df['pred_claim_number'] = mdl.predict(X)

In [51]:
df['theoretical_new_poisson_lambda'] = df['poisson_lambda'] * df['remains_prob']

In [52]:
df.sample(5)

,cat_feature,poisson_lambda,raw_claim_number,cancel_prob,claim_cancellation,claim_number,remains_prob,log_remains_prob,pred_claim_number_v2,pred_claim_number,theoretical_new_poisson_lambda
43914,0,0.2,0,0.571715,[],0,0.428285,-0.847966,0.084932,0.084932,0.085657
38650,0,0.2,0,0.270350,[],0,0.729650,-0.315190,0.144845,0.144845,0.145930
68555,1,0.7,1,0.465824,[0.597074164227756],1,0.534176,-0.627029,0.372939,0.372939,0.373923
6933,0,0.2,1,0.306143,[0.483508064549653],1,0.693857,-0.365490,0.137726,0.137726,0.138771
74283,1,0.7,1,0.195319,[0.48901693794181345],1,0.804681,-0.217310,0.562242,0.562242,0.563277


OK, seems to work. 

# Severity

In [44]:
df = pd.DataFrame.from_dict({'feature_1' : np.random.choice(2,10000)})

df['shape'] = 120 + df['feature_1'] * 20
df['scale'] = 120
df['gross_claim_amount'] = np.random.gamma(shape = df['shape'], scale = df['scale'])

df['deductible'] = 15000
df['net_claim_amount'] = (df['gross_claim_amount'] - df['deductible']).clip(0,None)
df['claim_observed'] = df['net_claim_amount'] > 0

In [45]:
df.groupby(["feature_1", "claim_observed"]).agg(
    number_of_policies = ("gross_claim_amount", "size"))

number_of_policies
feature_1 claim_observed                    
0         False                         3422
          True                          1559
1         False                          494
          True                          4525

In [54]:
df_observed = df[df['claim_observed'] == True].copy()

## Ground Truth

This is the theoretical result we would ideally have, the model on gross_claim_amount. 

In [46]:
mdl = GammaRegressor(alpha = 0)
X = df[['feature_1']]
y = df['gross_claim_amount']
mdl.fit(X, y)
df['pred_gross_claim_amount_method_01'] = mdl.predict(X)

Gets it right, as expected.

## Fitting on Net Claims

What happens if we simply fit on the observed parts of the claims? And then add the gross values back later? 

In [57]:
mdl = GammaRegressor(alpha = 0)
X = df_observed[['feature_1']]
y = df_observed['net_claim_amount']
mdl.fit(X, y)
df_observed['pred_net_claim_amount_method_02'] = mdl.predict(X)
df_observed['pred_gross_claim_amount_method_02'] = \
    df_observed['pred_net_claim_amount_method_02'] + df_observed['deductible']

In [58]:
df_observed

,feature_1,shape,scale,gross_claim_amount,deductible,net_claim_amount,claim_observed,pred_gross_claim_amount_method_01,pred_gross_claim_amount_method_02,pred_net_claim_amount_method_02
1,1,140,120,18518.833070,15000,3518.833070,True,16819.197653,17081.157222,2081.157222
4,1,140,120,16516.413406,15000,1516.413406,True,16819.197653,17081.157222,2081.157222
5,1,140,120,18233.237507,15000,3233.237507,True,16819.197653,17081.157222,2081.157222
6,1,140,120,16869.515462,15000,1869.515462,True,16819.197653,17081.157222,2081.157222
7,1,140,120,17505.533092,15000,2505.533092,True,16819.197653,17081.157222,2081.157222
...,...,...,...,...,...,...,...,...,...,...
9992,0,120,120,15485.289389,15000,485.289389,True,14391.687267,15907.746628,907.746628
9993,1,140,120,16279.253473,15000,1279.253473,True,16819.197653,17081.157222,2081.157222
9996,1,140,120,17312.543758,15000,2312.543758,True,16819.197653,17081.157222,2081.157222
9998,1,140,120,21556.973427,15000,6556.973427,True,16819.197653,17081.157222,2081.157222


## Fitting on Gross Claims

I actually wonder which is going to be closer. 

In [59]:
mdl = GammaRegressor(alpha = 0)
X = df_observed[['feature_1']]
y = df_observed['gross_claim_amount']
mdl.fit(X, y)
df_observed['pred_net_claim_amount_method_03'] = mdl.predict(X)

In [60]:
df_observed

,feature_1,shape,scale,gross_claim_amount,deductible,net_claim_amount,claim_observed,pred_gross_claim_amount_method_01,pred_gross_claim_amount_method_02,pred_net_claim_amount_method_02,pred_net_claim_amount_method_03
1,1,140,120,18518.833070,15000,3518.833070,True,16819.197653,17081.157222,2081.157222,17081.142284
4,1,140,120,16516.413406,15000,1516.413406,True,16819.197653,17081.157222,2081.157222,17081.142284
5,1,140,120,18233.237507,15000,3233.237507,True,16819.197653,17081.157222,2081.157222,17081.142284
6,1,140,120,16869.515462,15000,1869.515462,True,16819.197653,17081.157222,2081.157222,17081.142284
7,1,140,120,17505.533092,15000,2505.533092,True,16819.197653,17081.157222,2081.157222,17081.142284
...,...,...,...,...,...,...,...,...,...,...,...
9992,0,120,120,15485.289389,15000,485.289389,True,14391.687267,15907.746628,907.746628,15908.168252
9993,1,140,120,16279.253473,15000,1279.253473,True,16819.197653,17081.157222,2081.157222,17081.142284
9996,1,140,120,17312.543758,15000,2312.543758,True,16819.197653,17081.157222,2081.157222,17081.142284
9998,1,140,120,21556.973427,15000,6556.973427,True,16819.197653,17081.157222,2081.157222,17081.142284


OK, actually, that makes sense, it will be basically the expected value I think. 

In [63]:
df_observed.groupby(["feature_1"]).agg(
    average_gross_claim_amount = ("gross_claim_amount", np.mean))

,average_gross_claim_amount
feature_1,
0,15907.753206
1,17081.162503


Yeah, it's going to be quite off. 

I guess it's an intersting result that it doesn't matter if we simply fit on the gross or the net and then add it back. At least in this case. 